In [1]:
%%capture
import notebookimport

ds= __import__("Report - Driver Splits")

In [2]:
import sqlite3
from IPython.display import HTML
import pandas as pd

#Set near zero display to zero
#Doesn't have any effect?
pd.set_option('display.chop_threshold', 0.001)

dbname='testdata/finland19a.db'
conn = sqlite3.connect(dbname)
rally='Finland'
rc='RC1'
YEAR = year=2019

In [3]:
q_drivers='SELECT `driver.code` AS code  FROM startlists st JOIN startlist_classes sc ON sc.entryid = st.entryid AND name="{}"'

q_classes='SELECT DISTINCT(name) AS name FROM startlists st JOIN startlist_classes sc ON sc.entryid = st.entryid'

q_stages= 'SELECT * FROM itinerary_stages WHERE status!="ToRun" ORDER BY Number'

In [4]:
import dakar_utils as dakar
import ipywidgets as widgets
from ipywidgets import interact

## How About More Control?

What about we let the user decide what to allow in each chart?

THink about adding some pace notes to the table. For example, a column showing what the pace was on the current stage in terms of s / km gained or lost, the pace required in the next stage to compensate for the gap to other drivers from the rebased driver, the pace required over the rest of the stages in the loop, or over the next loop.

In [5]:
classes2 = widgets.Dropdown(
    options=pd.read_sql(q_classes,conn)['name'].to_list(),
    value='RC1',
    description='Class:',
    disabled=False,
)

drivers2 = widgets.Dropdown(
    options=pd.read_sql(q_drivers.format(classes2.value),conn)['code'].to_list(),
    #value='NEU',
    description='Driver:',
    disabled=False,
)

stages2 = widgets.Dropdown(
    options=pd.read_sql(q_stages,conn)['code'].to_list(), description='Stage:', disabled=False)


def stage_chart2(rc,driver, stage, bars, roadPos, waypointRank, previous):
    dropcols=[]
    if not roadPos:
        dropcols.append('Road Position')
    if not waypointRank:
        dropcols.append('Waypoint Rank')
    if not previous:
        dropcols.append('Previous')
        
    s2 = ds.getDriverSplitsReport(conn, rally, stage, driver, rc,
                                  bars=bars, dropcols=dropcols)
    display(HTML(s2))

splitBars = widgets.Checkbox( value=True, description='Split bars:',
                           disabled=False )
    
roadPos = widgets.Checkbox( value=True, description='Road pos',
                           disabled=False )

waypointRank = widgets.Checkbox( value=True, description='Waypoint Rank',
                           disabled=False )

previous = widgets.Checkbox( value=True, description='Previous',
                           disabled=False )


def update_drivers2(*args):
    qdrivers = q_drivers.format(classes2.value)
    driverlist = pd.read_sql(qdrivers,conn)['code'].to_list()
    print(driverlist)
    drivers2.options = driverlist
    
classes2.observe(update_drivers2, 'value')


#Add sort order

interact(stage_chart2, rc=classes2, driver=drivers2,
         stage=stages2, bars=splitBars, roadPos=roadPos,
        waypointRank=waypointRank, previous=previous);

interactive(children=(Dropdown(description='Class:', options=('RC1', 'RC2', 'RC4', 'RC3', 'RC5'), value='RC1')…

In [6]:
import testRequiredPace as pace

In [9]:
pace.conn = conn
pace.rally = rally
pace.YEAR = year

pace.distances = pace.stageDistances(conn)

In [11]:
from dakar_utils import moreStyleDriverSplitReportBaseDataframe

driver='LAT'
tmp =  pace.multiStagePaceReport(['SS{}'.format(i) for i in range(1,24)], driver )

#The negative map means we get times as the rebased driver is concerned...
s2 = moreStyleDriverSplitReportBaseDataframe(-tmp.loc[,:],'')
display(HTML(s2))


Stage dist is 2.31 km
Stage dist is 19.34 km
Stage dist is 20.04 km
Stage dist is 12.28 km
Stage dist is 12.33 km
Stage dist is 7.8 km
Stage dist is 20.04 km
Stage dist is 12.28 km
Stage dist is 12.33 km
Stage dist is 7.8 km
Stage dist is 2.31 km
Stage dist is 14.42 km
Stage dist is 22.87 km
Stage dist is 18.7 km
Stage dist is 10.5 km
Stage dist is 14.42 km
Stage dist is 22.87 km
Stage dist is 18.7 km
Stage dist is 10.5 km
Stage dist is 11.75 km
Stage dist is 11.12 km
Stage dist is 11.75 km
Stage dist is 11.12 km


,SS1,SS2,SS3,SS4,SS5,SS6,SS7,SS8,SS9,SS10,SS11,SS12,SS13,SS14,SS15,SS16,SS17,SS18,SS19,SS20,SS21,SS22,SS23
drivercode,,,,,,,,,,,,,,,,,,,,,,,
TÄN,0.735931,-0.0465357,0.144711,-0.0977199,0.27575,-0.0641026,-0.289421,-0.0162866,-0.121655,-0.128205,0.21645,0.214979,-0.0306078,0.754011,-0.00952381,0.18724,0.25798,0.272727,0.12381,0.178723,-0.143885,0.0340426,0.318615
LAP,-0.30303,-0.00517063,-0.00998004,-0.0977199,0.154096,-0.153846,-0.00499002,0.0814332,-0.0405515,-0.179487,0.04329,0.0416089,-0.205509,0.320856,-0.171429,0.443828,0.170529,0.299465,-0.114286,-0.12766,-0.323741,0.110638,-0.0836331
LAT,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0
MIK,0.0865801,-0.258532,-0.0499002,-0.154723,0.154096,-0.346154,-0.244511,0.105863,-0.186537,-0.0641026,0.12987,-0.124827,-0.51596,0.294118,-0.304762,0.0901526,-0.0699606,0.165775,0.133333,-0.0510638,-0.0629496,0.144681,0.104047
OGI,0.47619,-0.217166,-0.0598802,-0.138436,0.0729927,-0.269231,-0.269461,-0.0814332,-0.162206,-0.0384615,0.34632,-0.409154,-0.349803,0.203209,-0.209524,0.298197,-0.0131176,-0.00534759,-0.0761905,0.0851064,-0.125899,0.102128,0.0596223
BRE,-0,-0.227508,-0.0449102,-0.0488599,0.186537,-0.0128205,-0.314371,-0.02443,-0.251419,-0.115385,0.04329,-0.291262,-0.367293,0.0534759,-0.228571,0.0554785,-0.0306078,-0.00534759,-0.2,-0.297872,-0.413669,-0.502128,-0.0561151
NEU,1.03896,-0.610134,-0.209581,-0.36645,0.0081103,-0.0641026,-0.414172,-0.187296,-0.178427,-0.115385,0.562771,-0.145631,-0.49847,0.192513,-0.438095,0.180305,-0.166157,-0.251337,0.0761905,-0.476596,-0.179856,-0.331915,0.256475
SUN,-0,-0.274043,-0.219561,-0.570033,-0.308191,-0.641026,-0.459082,-0.480456,-0.600162,-0.564103,-0.0865801,-0.603329,-0.555313,-0.176471,-0.552381,-0.256588,-0.161784,-0.101604,-0.390476,-0.706383,-0.701439,-0.4,-0.293975
GRE,-1.2987,-0.718718,-0.449102,-0.610749,-1.46796,-0.75641,-0.713573,-0.700326,-0.900243,-0.525641,-1.21212,-1.53953,-0.913861,-0.818182,-1.41905,-0.825243,-0.603411,-0.791444,-0.838095,-1.08085,0,0,0
